In [ ]:
!gmx --version

In [ ]:
%%bash

git clone https://github.com/m3g/packmol.git
cd packmol
git checkout v20.3.5
./configure
make

In [ ]:
!git clone https://github.com/zhmurov/trappeua
!mv trappeua/trappeua.ff .
!rm -rf trappeua

In [ ]:
!packmol/packmol < input/packmol.inp

In [ ]:
!mv input/topol.top .
!gmx editconf -f conf.pdb -o conf.gro -box 4 4 4
!gmx grompp -f input/em.mdp -c conf.gro -o em.tpr
!gmx mdrun -deffnm em

!gmx grompp -f input/nvt.mdp -c em.gro -o nvt.tpr
!gmx mdrun -deffnm nvt
!gmx grompp -f input/npt.mdp -c nvt.gro -o npt.tpr
!gmx mdrun -deffnm npt